In [1]:
!nvidia-smi

Fri Jun 23 08:57:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np
import pandas as pd
import os, gc
from tqdm.auto import tqdm
from datetime import datetime, timezone, timedelta
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset, RandomSampler, SequentialSampler
import datasets
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, PeftModel, TaskType, LoraConfig
import pickle

In [3]:
with open("/content/X_train.pkl", "rb") as f:
  X_train = pickle.load(f)

with open("/content/Y_train.pkl", "rb") as f:
  Y_train = pickle.load(f)

In [4]:
# inspect data
query = X_train[0]
sql_command = Y_train[0]
print(f"query: {query}")
print()
print(f"SQL: {sql_command}")

print("="*100)
query = X_train[11]
sql_command = Y_train[11]
print(f"query: {query}")
print()
print(f"SQL: {sql_command}")

query: 쿼리:정신 의료 기관인 곳의 이름과 전화번호는 뭐야. 테이블 이름: 경상북도 정신의료기관 등 현황 (STS_GB_PSYCHIATRIC_INST_ETC) 테이블 컬럼: 연번 (SERIAL_NUMJ);구분 (SORTATIONK);기관명 (NM_ORGANIZATIONL);주소 (ADDRESSM);전화번호 (PHONE_NUMN);

SQL: SELECT NM_ORGANIZATIONL, PHONE_NUMN FROM STS_GB_PSYCHIATRIC_INST_ETC WHERE SORTATIONK = '정신의료기관'
query: 쿼리:사업명이 노노케어이거나 유형에 공익이 들어가는 사업의 수행기관을 중복 없이 보여줘. 테이블 이름: 경기도 시흥시 노인일자리사업현황 (STS_GG_SU_JOB_PRO_ELDERLY) 테이블 컬럼: 수행기관 (PERFORMING_ORGANIZATIONN);유형 (TYPEO);사업명 (BUSINESS_NMP);배정인원 (ASSIGNED_NUM_PEOPLEQ);내용 (CONTENTR);데이터 기준일자 (DATA_BASE_DTS);

SQL: SELECT DISTINCT PERFORMING_ORGANIZATIONN FROM STS_GG_SU_JOB_PRO_ELDERLY WHERE BUSINESS_NMP = '노노케어' OR TYPEO LIKE '%공익%'


In [5]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-5.8b")

In [6]:
train_input_ids, train_attn_masks, train_labels = [], [], []

for i in tqdm(range(len(X_train)), desc="preprocessing data"):
  query = tokenizer(X_train[i])
  target = tokenizer(Y_train[i])
  input_ids = query["input_ids"] + target["input_ids"] + [tokenizer.eos_token_id]
  attention_mask = query["attention_mask"] + target["attention_mask"] + [1]
  labels = [-100] * len(query["input_ids"]) + target["input_ids"] + [tokenizer.eos_token_id]
  train_input_ids.append(input_ids)
  train_attn_masks.append(attention_mask)
  train_labels.append(labels)


preprocessing data:   0%|          | 0/40 [00:00<?, ?it/s]

In [7]:
max_len = 0
max_idx = -1
for i in range(len(train_input_ids)):
  if max_len < len(train_input_ids[i]):
    max_len = len(train_input_ids[i])
    max_idx = i

print(max_len, max_idx)

595 23


In [8]:
def left_pad(sequence, value, max_len):
  return [value] * (max_len - len(sequence)) + sequence

train_input_ids_, train_attn_masks_, train_labels_ = [], [], []

for i in tqdm(range(len(train_input_ids))):
  train_input_ids_.append(left_pad(train_input_ids[i], tokenizer.pad_token_id, max_len))
  train_attn_masks_.append(left_pad(train_attn_masks[i], 0, max_len))
  train_labels_.append(left_pad(train_labels[i], -100, max_len))

  0%|          | 0/40 [00:00<?, ?it/s]

In [9]:
train_input_ids_ = torch.tensor(train_input_ids_, dtype=int)
train_attn_masks_ = torch.tensor(train_attn_masks_, dtype=int)
train_labels_ = torch.tensor(train_labels_, dtype=int)

train_input_ids_.shape, train_attn_masks_.shape, train_labels_.shape

(torch.Size([40, 595]), torch.Size([40, 595]), torch.Size([40, 595]))

In [10]:
train_data = TensorDataset(train_input_ids_, train_attn_masks_, train_labels_)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=2)

In [11]:
base_model = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/polyglot-ko-5.8b", device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

In [12]:
peft_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    r = 8,
    lora_alpha = 32,
    lora_dropout = 0.1,
    target_modules = ["query_key_value"] # GPT-NeoX base
)

print(peft_config)

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, base_model_name_or_path=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=['query_key_value'], lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True)


In [13]:
peft_model = get_peft_model(base_model, peft_config)
peft_model.to("cuda")
peft_model.train()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 4096)
        (layers): ModuleList(
          (0-27): 28 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (attention): GPTNeoXAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): Linear(
                in_features=4096, out_features=12288, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=12288, bias=False)
                )
   

In [14]:
# check information
pd.DataFrame([
    (param.dtype, param.shape, param.device, param.requires_grad, name)
    for name, param in peft_model.named_parameters()
], columns=["dtype", "shape", "device", "requires_grad", "name"])

,dtype,shape,device,requires_grad,name
0,torch.float32,"(30080, 4096)",cuda:0,False,base_model.model.gpt_neox.embed_in.weight
1,torch.float32,"(4096,)",cuda:0,False,base_model.model.gpt_neox.layers.0.input_layer...
2,torch.float32,"(4096,)",cuda:0,False,base_model.model.gpt_neox.layers.0.input_layer...
3,torch.float32,"(4096,)",cuda:0,False,base_model.model.gpt_neox.layers.0.post_attent...
4,torch.float32,"(4096,)",cuda:0,False,base_model.model.gpt_neox.layers.0.post_attent...
...,...,...,...,...,...
391,torch.float32,"(4096, 16384)",cuda:0,False,base_model.model.gpt_neox.layers.27.mlp.dense_...
392,torch.float32,"(4096,)",cuda:0,False,base_model.model.gpt_neox.layers.27.mlp.dense_...
393,torch.float32,"(4096,)",cuda:0,False,base_model.model.gpt_neox.final_layer_norm.weight
394,torch.float32,"(4096,)",cuda:0,False,base_model.model.gpt_neox.final_layer_norm.bias


In [ ]:
train_losses = []
learning_rate = 3e-5
optimizer = torch.optim.AdamW(peft_model.parameters(), lr=learning_rate)
num_epochs = 10
peft_model.train()
for epoch in range(num_epochs):
  train_loss = 0
  tr_loss = torch.tensor(0.0).to("cuda")
  for batch_idx, batch in enumerate(tqdm(train_dataloader), start=1):
    optimizer.zero_grad()
    batch = tuple(t.to("cuda") for t in batch)
    b_input_ids, b_attn_masks, b_labels = batch
    outputs = peft_model(input_ids=b_input_ids, attention_mask=b_attn_masks, labels=b_labels)
    step_loss = outputs[0]
    step_loss.backward()
    optimizer.step()
    tr_loss += step_loss.detach()
  avg_train_loss = tr_loss / len(train_dataloader)
  train_losses.append(avg_train_loss)
  print(f"epoch: {epoch+1} | average train loss: {avg_train_loss}")

  0%|          | 0/20 [00:00<?, ?it/s]

epoch: 1 | average train loss: 0.6836880445480347


  0%|          | 0/20 [00:00<?, ?it/s]

epoch: 2 | average train loss: 0.4941668212413788


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

epoch: 4 | average train loss: 0.21147988736629486


  0%|          | 0/20 [00:00<?, ?it/s]

epoch: 5 | average train loss: 0.13057644665241241


  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
# save model
TIME_SERIAL = datetime.now(timezone(timedelta(hours=9))).strftime("%y%m%d-%H%M%S")
PEFT_MODEL_PATH = f"GPTNEOX_NL2SQL_POC_{TIME_SERIAL}"
peft_model.save_pretrained(PEFT_MODEL_PATH)

# loaded_model = PeftModel.from_pretrained(model=base_model, model_id=PEFT_MODEL_PATH)

In [ ]:
sample_query = "쿼리:관악구에 있고 공립인 특수학교의 영문학교명과 홈페이지 주소를 알려줘. 테이블 이름: 서울시 특수학교 기본정보 (NEIS_SCHOOL_INFO_DS) 테이블 컬럼: 시도교육청코드 (ATPT_OFCDC_SC_CODE);시도교육청명 (ATPT_OFCDC_SC_NM);표준학교코드 (SD_SCHUL_CODE);학교명 (SCHUL_NM);영문학교명 (ENG_SCHUL_NM);학교종류명 (SCHUL_KND_SC_NM);소재지명 (LCTN_SC_NM);설립명 (FOND_SC_NM);도로명우편번호 (ORG_RDNZC);도로명주소 (ORG_RDNMA);도로명상세주소 (ORG_RDNDA);홈페이지주소 (HMPG_ADRES);팩스번호 (ORG_FAXNO);산업체특별학급존재여부 (INDST_SPECL_CCCCL_EXST_YN);고등학교일반실업구분명 (HS_GNRL_BUSNS_SC_NM);입시전후기구분명 (ENE_BFE_SEHF_SC_NM);개교기념일 (FOAS_MEMRD);적재일시 (LOAD_DTM);"

sample_answer = "SELECT ENG_SCHUL_NM, HMPG_ADRES FROM NEIS_SCHOOL_INFO_DS WHERE FOND_SC_NM = '공립' AND ORG_RDNMA LIKE '%관악구%'"

In [ ]:
query = tokenizer(sample_query)
sample_input_ids = query["input_ids"]
sample_attention_mask = query["attention_mask"]

sample_input_ids = torch.tensor(sample_input_ids,dtype=int)
sample_input_ids = torch.reshape(sample_input_ids, (1, -1))
sample_input_ids = sample_input_ids.to("cuda")

sample_attention_mask = torch.tensor(sample_attention_mask, dtype=int)
sample_attention_mask = torch.reshape(sample_attention_mask, (1, -1))
sample_attention_mask = sample_attention_mask.to("cuda")

In [ ]:
peft_model.eval()
with torch.no_grad():
  generated = peft_model.generate(input_ids=sample_input_ids,
                                  attention_mask=sample_attention_mask,
                                  pad_token_id=tokenizer.pad_token_id,
                                  max_new_tokens=100,
                                  do_sample=False,
                                  num_beams=1,
                                  num_beam_groups=1,
                                  penalty_alpha=None,
                                  use_cache=True,
                                  temperature=0.7)

response = tokenizer.decode(generated[:, sample_input_ids.shape[-1]:][0], skip_special_tokens=True)
print(response)